In [13]:
%autoreload 2

UsageError: Line magic function `%autoreload` not found.


In [14]:
import os
import anndata as ad
import torch
import numpy as np

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

os.chdir("/home/ico/PycharmProjects/SCOOTR/run")
os.getcwd()

'/home/ico/PycharmProjects/SCOOTR/run'

In [23]:
USE_ADT2GEX = False
if USE_ADT2GEX:
    #adt2gex
    test_path = "../datasets/openproblems_bmmc_cite_phase2_rna/openproblems_bmmc_cite_phase2_rna.censor_dataset.output_"
else:
    #atac2gex
    test_path = "../datasets/openproblems_bmmc_multiome_phase2_rna/openproblems_bmmc_multiome_phase2_rna.censor_dataset.output_"

prediction_path = "../pretrain/defaultGEX2ATAC.h5ad"

par = {
    "input_test_prediction": prediction_path,
    "input_test_sol": f"{test_path}test_sol.h5ad",
}

prediction_test = ad.read_h5ad(par["input_test_prediction"])
sol_test = ad.read_h5ad(par["input_test_sol"])

In [24]:
if type(prediction_test.X) != np.ndarray:
    X = prediction_test.X.toarray()
else:
    X = prediction_test.X
X = torch.tensor(X)

Xsol = torch.tensor(sol_test.X.toarray())
Xsol.argmax(1)
# Order the columns of the prediction matrix so that the perfect prediction is the identity matrix
X = X[:, Xsol.argmax(1)]

In [25]:
labels = torch.arange(X.shape[0])
forward_accuracy = (torch.argmax(X, dim=1) == labels).float().mean().item()
backward_accuracy = (
    (torch.argmax(X, dim=0) == labels).float().mean().item()
)
avg_accuracy = 0.5 * (forward_accuracy + backward_accuracy)
print(forward_accuracy, backward_accuracy, "top1-acc:", avg_accuracy)

0.07756509631872177 0.08036383986473083 top1-acc: 0.0789644680917263


In [26]:
_, top_indexes_forward = X.topk(5, dim=1)
_, top_indexes_backward = X.topk(5, dim=0)
l_forward = labels.expand(5, X.shape[0]).T
l_backward = l_forward.T
top5_forward_accuracy = (
    torch.any(top_indexes_forward == l_forward, 1).float().mean().item()
)
top5_backward_accuracy = (
    torch.any(top_indexes_backward == l_backward, 0).float().mean().item()
)
top5_avg_accuracy = 0.5 * (top5_forward_accuracy + top5_backward_accuracy)

print(top5_forward_accuracy, top5_backward_accuracy, "top5-acc:", top5_avg_accuracy)

0.19691139459609985 0.1980108916759491 top5-acc: 0.19746114313602448


In [7]:
top_indexes_forward

tensor([[ 2102,  9328, 12239,  3118, 14941],
        [13537,  9475,  3363,  2144, 14055],
        [13127, 13084, 12243,  9073,  3017],
        ...,
        [ 9517,  4774, 12743, 12597,  4924],
        [15064, 10859, 14854, 13326, 10268],
        [ 2897,  2844,  3640,  4628,   492]])

In [8]:
(top_indexes_forward[:, 1:] != torch.Tensor([1,3,4,0])).float().sum()

tensor(60259.)

In [9]:
torch.any(top_indexes_forward == l_forward, 1).float().mean()

tensor(0.1302)

### FOSCTTM

In [27]:
(X > torch.diag(X)).float().mean().item()


0.00840053241699934

In [11]:

foscttm_x = (X >= torch.diag(X)).float().mean(axis=1)
foscttm_y = (X >= torch.diag(X)).float().mean(axis=0)
# foscttm_y = (d < np.expand_dims(np.diag(d), axis=0)).mean(axis=0)
print("foscttm_x", foscttm_x, "foscttm_y", foscttm_y)

foscttm_x tensor([0.0082, 0.0183, 0.0078,  ..., 0.0047, 0.0125, 0.0114]) foscttm_y tensor([0.0056, 0.0033, 0.0011,  ..., 0.0057, 0.0002, 0.0239])


In [12]:
foscttm_y.mean()

tensor(0.0110)

### For soft predictions, the competition score can be made equal to the forward accuracy (or backward accuracy) by putting 1 at the max of each row (or each column) and 0 elsewhere

In [20]:
logits_row_sums = X.clip(min=0).sum(dim=1)
top1_competition_metric = X.clip(min=0).diag().div(logits_row_sums).mean().item()
top1_competition_metric

0.05983518436551094

In [22]:
mx = torch.max(X, dim=1, keepdim=True).values
hard_X = (mx == X).float()
logits_row_sums = hard_X.clip(min=0).sum(dim=1)
top1_competition_metric = hard_X.clip(min=0).diagonal().div(logits_row_sums).mean().item()
top1_competition_metric

0.10998274385929108

In [21]:
X.sum()

tensor(3.)